In [1]:
from bs4 import BeautifulSoup
import urllib.request
import urllib.parse
import pandas as pd
import requests
import numpy as np
from tqdm import tqdm_notebook 

In [8]:
# paginate = soup.find("div",{"class":"paginate"})
# max_page = list(paginate.get_text().replace("\n"," ").split())
# if '>' not in max_page:
#     print("대따")

In [2]:
#끝 페이지까지 가는 구현 부분입니다
page = 1
while 1:
    url = f"https://www.hanbit.co.kr/academy/books/new_book_list.html?page={page}&cate_cd=&srt=&searchKey=&keyWord="
    with urllib.request.urlopen(url) as res:
        html = res.read()
        soup = BeautifulSoup(html,'html.parser')
    paginate = soup.find("div",{"class":"paginate"})
    page_list = list(paginate.get_text().replace("\n"," ").split())
    if '>' not in page_list:
        break
    page = page + 10
max_page = int(page_list[-1])
print(max_page)

16


In [3]:
for page in tqdm_notebook(range(max_page)):
    basepage = 1+page
    url = f"https://www.hanbit.co.kr/academy/books/new_book_list.html?page={basepage}&cate_cd=&srt=&searchKey=&keyWord="
    with urllib.request.urlopen(url) as res:
        html = res.read()
        soup = BeautifulSoup(html,'html.parser')
    #1페이지일경우 리스트 초기화
    if basepage == 1:
        titles=[]
        writers=[] 
        prices=[]
    #class info로 이름과 저자를 동시에 가져옴
    souce = soup.find_all("div",{"class":"info"})
    price = soup.find_all("span",{"class":"price"})
    for i in souce:
        title_writer = i.get_text().split('\n')
        titles.append(title_writer[1])
        writers.append(title_writer[2])
    for i in price:
        prices.append(int(i.get_text()[:-1].replace(",","")))
book = pd.DataFrame({
            "title":titles,
            "writer":writers,
            "price":prices,},
            index=np.arange(1,len(titles)+1))
            #index부분이 1부터 시작
book

,title,writer,price
1,쉽게 배우는 자바 프로그래밍 2판,우종정,29000
2,포르잔 C++ 에센셜,"Behrouz A. Forouzan , Richard F. Gilberg",32000
3,"IT CookBook, 액세스 2019로 배우는 데이터베이스 기초와 실습",김지연,28000
4,포르잔 C++ 바이블,"Behrouz A. Forouzan , Richard F. Gilberg",45000
5,딥러닝을 위한 선형대수학,Gilbert Strang,38000
...,...,...,...
298,문제해결력을 키우는 이공계 글쓰기 : 발표와 토론까지,한성우,18000
299,"진로개발과 직업세계 : 대학생과 청년층을 위한 진로, 취업지침서","서일수 , 김도균 , 김지혜 , 이정배",20000
300,Essential Calculus,James Stewart,36000
301,"IT CookBook, 디지털 통신 시스템 : Digital Communicatio...","정호영 , 임승각",25000


In [14]:
book.to_csv("과제.csv")

In [85]:
trans=[]
dates=[]
pages = []
isbns = []
Logistics_code=[]
page=1
url = f"https://www.hanbit.co.kr/academy/books/new_book_list.html?page={page}&cate_cd=&srt=&searchKey=&keyWord="
with urllib.request.urlopen(url) as res:
    html = res.read()
    soup = BeautifulSoup(html,'html.parser')
link = soup.select('span.pop_quick_menu > a[href]')[0::2]
for i in tqdm_notebook(range(len(link))[:4]):
    han_url = "https://www.hanbit.co.kr/"
    detail_url = link[i].attrs['href']
    with urllib.request.urlopen(han_url + detail_url) as res:
        html = res.read()
        detail = BeautifulSoup(html,'html.parser')
    detail_list = detail.select("ul.info_list > li")
    if detail_list[1].get_text()[:3].find("번역") >= 0:
        trans.append(detail_list[1].get_text()[5:])
        dates.append(detail_list[2].get_text()[5:])
        pages.append(detail_list[3].get_text()[5:-1])
        isbns.append(detail_list[4].get_text()[7:])
        Logistics_code.append(detail_list[5].get_text()[6:])
    else:
        trans.append(' ')
        dates.append(detail_list[1].get_text()[5:])
        pages.append(detail_list[2].get_text()[5:-1])
        isbns.append(detail_list[3].get_text()[7:])
        Logistics_code.append(detail_list[4].get_text()[6:])
        trans.append(' ')
    # trans.append(detail_list[1].get_text()[5:] if detail_list[1].get_text()[:3]
    #dates.append(detail_list[2].get_text()[5:] if detail_list[1].get_text()[:3].find("출간") >= 0 else ' ')
print(detail_list)
    # print(detail_list[1].get_text())
    # print(detail_list[0].find("span").get_text())


[<li><strong>저자 : </strong><span>Gilbert Strang </span></li>, <li><strong>번역 : </strong><span>부산대학교 산업수학센터(FFMIMC) </span></li>, <li><strong>출간 : </strong><span>2020-08-05</span></li>, <li><strong>페이지 : </strong><span>560 쪽</span></li>, <li><strong>ISBN : </strong><span>9791156644972</span></li>, <li><strong>물류코드 :</strong><span>4497</span></li>]


In [106]:
for page in tqdm_notebook(range(max_page)):
    basepage = 1+page
    url = f"https://www.hanbit.co.kr/academy/books/new_book_list.html?page={basepage}&cate_cd=&srt=&searchKey=&keyWord="
    with urllib.request.urlopen(url) as res:
        html = res.read()
        soup = BeautifulSoup(html,'html.parser')
    #1페이지일경우 리스트 초기화
    if basepage == 1:
        titles=[]
        writers=[] 
        prices=[]
        trans=[]
        dates=[]
        pages = []
        isbns = []
        Logistics_code=[]

    #페이지 상세보기에서 상세정보 가져오기
    link = soup.select('span.pop_quick_menu > a[href]')[0::2]
    for i in range(len(link)):
        han_url = "https://www.hanbit.co.kr/"
        detail_url = link[i].attrs['href']
        with urllib.request.urlopen(han_url + detail_url) as res:
            html = res.read()
            detail = BeautifulSoup(html,'html.parser')
        detail_list = detail.select("ul.info_list > li")
        #가져온 데이터 편집해서 list에 집어 넣기
        if detail_list[1].get_text()[:3].find("번역") >= 0:
            trans.append(detail_list[1].get_text()[5:-1])
            dates.append(detail_list[2].get_text()[5:])
            pages.append(detail_list[3].get_text()[5:-1])
            isbns.append(detail_list[4].get_text()[7:])
            Logistics_code.append(detail_list[5].get_text()[6:])
        else:
            trans.append(' ')
            dates.append(detail_list[1].get_text()[5:])
            pages.append(detail_list[2].get_text()[5:-1])
            isbns.append(detail_list[3].get_text()[7:])
            Logistics_code.append(detail_list[4].get_text()[6:])
    #class info로 이름과 저자를 동시에 가져옴
    souce = soup.find_all("div",{"class":"info"})
    price = soup.find_all("span",{"class":"price"})
    for i in souce:
        title_writer = i.get_text().split('\n')
        titles.append(title_writer[1])
        writers.append(title_writer[2])
    for i in price:
        prices.append(int(i.get_text()[:-1].replace(",","")))
book = pd.DataFrame({
            "title":titles,
            "writer":writers,
            "price":prices,
            'trans':trans,
            'dates':dates,
            'pages':pages,
            'isbns':isbns,
            'Logistics_code':Logistics_code},
            index=np.arange(1,len(titles)+1))
            #index부분이 1부터 시작
book

,title,writer,price,trans,dates,pages,isbns,Logistics_code
1,포르잔 C++ 에센셜,"Behrouz A. Forouzan , Richard F. Gilberg",32000,윤인성,2020-11-10,756,9791156645061,4506
2,"IT CookBook, 액세스 2019로 배우는 데이터베이스 기초와 실습",김지연,28000,,2020-09-28,536,9791156645054,4505
3,포르잔 C++ 바이블,"Behrouz A. Forouzan , Richard F. Gilberg",45000,윤인성,2020-08-31,1192,9791156645030,4503
4,딥러닝을 위한 선형대수학,Gilbert Strang,38000,부산대학교 산업수학센터(FFMIMC),2020-08-05,560,9791156644972,4497
5,"IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습(2판)","박우창 , 남송휘 , 이현룡",29000,,2020-07-30,600,9791156645023,4502
...,...,...,...,...,...,...,...,...
297,문제해결력을 키우는 이공계 글쓰기 : 발표와 토론까지,한성우,18000,,2013-02-28,396,9788998756024,4002
298,"진로개발과 직업세계 : 대학생과 청년층을 위한 진로, 취업지침서","서일수 , 김도균 , 김지혜 , 이정배",20000,,2013-02-28,392,9788998756048,4004
299,Essential Calculus,James Stewart,36000,,2012-07-08,960,9781133492566,7999
300,"IT CookBook, 디지털 통신 시스템 : Digital Communicatio...","정호영 , 임승각",25000,,2011-01-31,576,9788979147988,1798


In [90]:
book.to_csv("과제.csv")

In [107]:
book[book['trans']=="윤인성"]

,title,writer,price,trans,dates,pages,isbns,Logistics_code
1,포르잔 C++ 에센셜,"Behrouz A. Forouzan , Richard F. Gilberg",32000,윤인성,2020-11-10,756,9791156645061,4506
3,포르잔 C++ 바이블,"Behrouz A. Forouzan , Richard F. Gilberg",45000,윤인성,2020-08-31,1192,9791156645030,4503


In [ ]:
성